# Global Alignment

Name: Yanfang Guo           
Department : MACS       
Email: Yanfang.Guo@vub.be         



## Using Matrix class to represent the BLOSOM matrix
- the blosom data is stored in "BLOSOM62.txt","BLOSOM50.txt","BLOSOM80.txt","BLOSOM90.txt", each time choose a proper blosom matrix
- inside the Matrix class, the main data structure is Matrix which use dictionary in Python
- initialize the Matrix in this way, Matrix("BLOSOM62").

In [1]:
class Matrix(object):
    def __init__(self,file):
        f = open(file)
        self.matrix ={}
        # c is the name of the column
        c = f.readline().split()
        for i in f:
            r = i.split()
            rowname = r.pop(0)
            t = {}
            for j in range(len(c)):
                t[c[j]] = int(r[j])
            self.matrix[rowname] = t

    def get(self,row,col):
        return self.matrix[row][col]

    def print(self):
        for key in self.matrix:
            for key2 in self.matrix[key]:

                print("{0:3d}".format(self.matrix[key][key2]),end="")

            print()

mat1 = Matrix("BLOSUM62.txt")
mat1.print()

  4 -1 -2 -2  0 -1 -1  0 -2 -1 -1 -1 -1 -2 -1  1  0 -3 -2  0 -2 -1  0 -4
 -1  5  0 -2 -3  1  0 -2  0 -3 -2  2 -1 -3 -2 -1 -1 -3 -2 -3 -1  0 -1 -4
 -2  0  6  1 -3  0  0  0  1 -3 -3  0 -2 -3 -2  1  0 -4 -2 -3  3  0 -1 -4
 -2 -2  1  6 -3  0  2 -1 -1 -3 -4 -1 -3 -3 -1  0 -1 -4 -3 -3  4  1 -1 -4
  0 -3 -3 -3  9 -3 -4 -3 -3 -1 -1 -3 -1 -2 -3 -1 -1 -2 -2 -1 -3 -3 -2 -4
 -1  1  0  0 -3  5  2 -2  0 -3 -2  1  0 -3 -1  0 -1 -2 -1 -2  0  3 -1 -4
 -1  0  0  2 -4  2  5 -2  0 -3 -3  1 -2 -3 -1  0 -1 -3 -2 -2  1  4 -1 -4
  0 -2  0 -1 -3 -2 -2  6 -2 -4 -4 -2 -3 -3 -2  0 -2 -2 -3 -3 -1 -2 -1 -4
 -2  0  1 -1 -3  0  0 -2  8 -3 -3 -1 -2 -1 -2 -1 -2 -2  2 -3  0  0 -1 -4
 -1 -3 -3 -3 -1 -3 -3 -4 -3  4  2 -3  1  0 -3 -2 -1 -3 -1  3 -3 -3 -1 -4
 -1 -2 -3 -4 -1 -2 -3 -4 -3  2  4 -2  2  0 -3 -2 -1 -2 -1  1 -4 -3 -1 -4
 -1  2  0 -1 -3  1  1 -2 -1 -3 -2  5 -1 -3 -1  0 -1 -3 -2 -2  0  1 -1 -4
 -1 -1 -2 -3 -1  0 -2 -3 -2  1  2 -1  5  0 -2 -1 -1 -1 -1  1 -3 -1 -1 -4
 -2 -3 -3 -3 -2 -3 -3 -3 -1  0  0 -3  0  6 -4 -2 -2

    

## Define I and E 

- I is opening gap penalty
- E is extending gap penalty
- if you want to use linear gap penalty, simply let I=E

In [2]:
I = -6
E = -4

## Read fasta file
- each time this method will read the fasta file and return a list with all the sequences

In [3]:
def read_fasta(filename):
    f = open(filename)
    proteins = []
    d = f.read().split(">")

    # deal with the  "" before the first ""
    d.pop(0)
    for i in d:
        t = i.splitlines()
        # get rid of the line of the protein name
        t.pop(0)
        s = "".join(t)
        proteins.append(s)
    return proteins
seq_list = read_fasta("WW-sequence.fasta")
seq1 =seq_list[0]
seq2 =seq_list[1]

for i in seq_list:
    print(i)

VPLPAGWEMAKTSSGQRYFLNHIDQTTTWQDPRK
SPLPPGWEERQDILGRTYYVNHESRRTQWKRPTP
GPLPPGWEERTHTDGRIFYINHNIKRTQWEDPRL
EKLPPGWEKRMSRSSGRVYYFNHITNASQWERPSG
SGAKSMWTEHKSPDGRTYYYNTETKQSTWEKPDD
LLSKCPWKEYKSDSGKPYYYNSQTKESRWAKPKE


## Here we initialize the direc_mat 
- direc_mat is an i*j Matrix(i,j are the length of two sequences)
- the element in the Matrix is a list with the length 5. The first 3 positions indicate the traceback direction(Vertical, horizental, Oblique), and the last two positions are used to indicate the next element of the penalty is opening gap penalty or extending gap penalty(in vertical and horizental directions)
- such as
 - t =[0,1,0,I,E]    
  t[0] t[1] t[2] indicates the direction of traceback.    
  t[3] t[4] use to deal with the gap penalty.     
  if t[0]=1 and t[3] is the first element of a gap, then t[3]=I(opening gap penalty), else t[3]=E.    
  the same with t[4]

In [4]:
direc_mat = []
# print_mat3, the function to print the direc_mat in a formative way.
def print_mat3(alist,n=3):
    for i in alist:
        for j in i:
            print(j[0:n],end='')
        print()

for i in range(len(seq1) + 1):
    # default, opening gap
    direc_mat.append([[0, 0, 0,I,I] for i in range(len(seq2) + 1)])

## Score_mat

In [5]:
score_mat = []
for i in range(len(seq1) + 1):
    score_mat.append([0 for i in range(len(seq2) + 1)])
    
# nicely output the score_mat
def print_mat2(alist):
    for i in alist:
        for j in i:
            print('{0:4d}'.format(j), end=" ")
        print()

## Initialize the score_mat and direc_mat

here we focused on the first line and the first row.


In [6]:
score_mat[0][0] = 0

direc_mat[0][1] = [0,1,0,I,E]
score_mat[0][1]=I
direc_mat[1][0]=[1,0,0,E,I]
score_mat[1][0]=I

for i in range(2, len(seq2)+1):
    direc_mat[0][i] = [0,1,0,I,E]
    score_mat[0][i] = score_mat[0][i-1] + direc_mat[0][i-1][4]
for i in range(2, len(seq1)+1):
    direc_mat[i][0] = [1,0,0,E,I]
    score_mat[i][0] = score_mat[i-1][0]+ direc_mat[i-1][0][3]
    
#print_mat2(score_mat)

## Calculate the score
calculate the score according to the formula given

In [7]:
def score2(mat):
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):

            t1 = score_mat[i - 1][j] + direc_mat[i-1][j][3]

            t2 = score_mat[i][j - 1] +direc_mat[i][j-1][4]

            t3 = score_mat[i - 1][j - 1] + mat.get(seq1[i - 1], seq2[j - 1])

            maxscore = max(t1, t2, t3)
            score_mat[i][j] = maxscore
            if (t1 == maxscore):
                direc_mat[i][j][1] = 1
                # affine gap
                direc_mat[i][j][3] = E
            if (t2 == maxscore):
                direc_mat[i][j][0] = 1
                direc_mat[i][j][4] = E
            if (t3 == maxscore):
                direc_mat[i][j][2] = 1
    return score_mat[len(seq1)][len(seq2)]

print(score2(mat1))

77


In [8]:
# print out the direc_mat(direc_mat[i][j][:3])
#print_mat3(direc_mat)

## Trace back and print out the route
- traceback accoording to the direc_mat and find path
- the function traceback have one parameter k, which specify the maxmum subpath
- the traceback function starts from the position[len(seq1)][len(seq2)] to [0][0]
- use the broad first search, every time we go one step ,and we add all the poss

In [9]:
def traceback(k=4):
    path_pair = []
    path_up = []
    path_down = []
    mark1 =[]
    mark2 =[]

    # use queue to trace multiple path

    queue = []
    queue.append([path_up[0:len(path_up)], path_down[0:len(path_down)], len(seq1), len(seq2)])
    while(len(queue)>0):
        t = queue.pop(0)
        i = t[2]
        j = t[3]
        path_up = t[0]
        path_down=t[1]

        if i ==0 or j ==0:
            if j==0 and i ==0 :
                path_pair.append([path_up,path_down])
            elif i ==0:
                for m in range(j+1,-1,-1):
                    path_up.insert(0,'-')
                    path_down.insert(0,seq2[m])
                path_pair.append([path_up,path_down])
            else:
                for m in range(i+1,-1,-1):
                    path_down.insert(0,'-')
                    path_up.insert(0,seq1[m])
                path_pair.append([path_up,path_down])
        #scan all possible path and append it to the queue
        else:
            if direc_mat[i][j][0] == 1:
                path_up.insert(0, '-')
                j = j - 1

                path_down.insert(0, seq2[j])
                if len(queue)<k :
                    queue.append([path_up[0:len(path_up)], path_down[0:len(path_down)], i, j])
                j = j+1
                path_up = path_up[1:]
                path_down = path_down[1:]
            if direc_mat[i][j][1] == 1:
                path_down.insert(0, '-')
                i = i - 1
                path_up.insert(0, seq1[i])
                if len(queue)<k :
                    queue.append([path_up[0:len(path_up)], path_down[0:len(path_down)], i, j])
                i = i+1
                path_up = path_up[1:]
                path_down = path_down[1:]

            if direc_mat[i][j][2] == 1:
                i = i - 1
                j = j - 1
                path_up.insert(0, seq1[i])
                path_down.insert(0, seq2[j])

                if len(queue)<k :
                    queue.append([path_up[0:len(path_up)], path_down[0:len(path_down)], i, j])
                i = i+1
                j = j+1
                path_up = path_up[1:]
                path_down = path_down[1:]



    return path_pair


def print_pathpair(alist):
    for path in alist:
        mark1=[]
        mark2=[]
        for i in range(len(path[0])):
            if path[0][i]!='-' and path[1][i]!='-':
                if path[0][i] == path[1][i]:
                    mark1.append(':')

                elif mat1.get(path[0][i],path[1][i])>0:
                    mark1.append('.')

                else:
                    mark1.append(' ')

            else:
                mark1.append(' ')


        path_u = ''.join(path[0])
        path_m = ''.join(mark1)


        path_d = ''.join(path[1])


        print(path_u)
        print(path_m)
        print(path_d)

In [10]:
print("score:",score2(mat1))
print_pathpair(traceback(2))

score: 77
VPLPAGWEMAKTSSGQRYFLNHIDQTTTWQDPRK
 ::: :::  .   :. :..::  . : :. :  
SPLPPGWEERQDILGRTYYVNHESRRTQWKRPTP


![](https://ww3.sinaimg.cn/large/006tNbRwgy1fdo3f3fw9lj311i07sta7.jpg)
the result and score given is exactly the same with the result in the website [LALIGN](http://embnet.vital-it.ch/software/LALIGN_form.html)

## Put all sub function into one method
- here for global method, we have the function fun(seq1,seq2,mat=mat1), this function encapsule all the sub function include score and traceback

In [11]:
def score2(mat,seq1,seq2,score_mat,direc_mat):
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):

            t1 = score_mat[i - 1][j] + direc_mat[i-1][j][3]

            t2 = score_mat[i][j - 1] +direc_mat[i][j-1][4]

            t3 = score_mat[i - 1][j - 1] + mat.get(seq1[i - 1], seq2[j - 1])

            maxscore = max(t1, t2, t3)
            score_mat[i][j] = maxscore
            if (t1 == maxscore):
                direc_mat[i][j][1] = 1
                # affine gap
                direc_mat[i][j][3] = E
            if (t2 == maxscore):
                direc_mat[i][j][0] = 1
                direc_mat[i][j][4] = E
            if (t3 == maxscore):
                direc_mat[i][j][2] = 1
    return score_mat[len(seq1)][len(seq2)]


def print_mat2(alist):
    for i in alist:
        for j in i:
            print('{0:4d}'.format(j), end=" ")
        print()


def print_mat3(alist):
    for i in alist:
        print(i)


def traceback(direc_mat,seq1,seq2,k=4):
    path_pair = []
    path_up = []
    path_down = []
    mark1 =[]
    mark2 =[]
    
    queue = []
    queue.append([path_up[0:len(path_up)], path_down[0:len(path_down)], len(seq1), len(seq2)])
    while(len(queue)>0):
        t = queue.pop(0)
        i = t[2]
        j = t[3]
        path_up = t[0]
        path_down=t[1]

        if i ==0 or j ==0:
            if j==0 and i ==0 :
                path_pair.append([path_up,path_down])
            elif i ==0:
                for m in range(j+1,-1,-1):
                    path_up.insert(0,'-')
                    path_down.insert(0,seq2[m])
                path_pair.append([path_up,path_down])
            else:
                for m in range(i+1,-1,-1):
                    path_down.insert(0,'-')
                    path_up.insert(0,seq1[m])
                path_pair.append([path_up,path_down])
        #scan all possible path and append it to the queue
        else:
            if direc_mat[i][j][0] == 1:
                path_up.insert(0, '-')
                j = j - 1

                path_down.insert(0, seq2[j])
                if len(queue)<k :
                    queue.append([path_up[0:len(path_up)], path_down[0:len(path_down)], i, j])
                j = j+1
                path_up = path_up[1:]
                path_down = path_down[1:]
            if direc_mat[i][j][1] == 1:
                path_down.insert(0, '-')
                i = i - 1
                path_up.insert(0, seq1[i])
                if len(queue)<k :
                    queue.append([path_up[0:len(path_up)], path_down[0:len(path_down)], i, j])
                i = i+1
                path_up = path_up[1:]
                path_down = path_down[1:]

            if direc_mat[i][j][2] == 1:
                i = i - 1
                j = j - 1
                path_up.insert(0, seq1[i])
                path_down.insert(0, seq2[j])

                if len(queue)<k :
                    queue.append([path_up[0:len(path_up)], path_down[0:len(path_down)], i, j])
                i = i+1
                j = j+1
                path_up = path_up[1:]
                path_down = path_down[1:]
    return path_pair

def fun(seq1,seq2):
    score_mat = []
    direc_mat = []
    for i in range(len(seq1) + 1):
        # default, opening gap
        direc_mat.append([[0, 0, 0, I, I] for i in range(len(seq2) + 1)])

    for i in range(len(seq1) + 1):
        score_mat.append([0 for i in range(len(seq2) + 1)])
    # initialize
    score_mat[0][0] = 0

    direc_mat[0][1] = [0, 1, 0, I, E]
    score_mat[0][1] = I
    direc_mat[1][0] = [1, 0, 0, E, I]
    score_mat[1][0] = I

    for i in range(2, len(seq2) + 1):
        direc_mat[0][i] = [0, 1, 0, I, E]
        score_mat[0][i] = score_mat[0][i - 1] + direc_mat[0][i - 1][4]
    for i in range(2, len(seq1) + 1):
        direc_mat[i][0] = [1, 0, 0, E, I]
        score_mat[i][0] = score_mat[i - 1][0] + direc_mat[i - 1][0][3]

    print("score:",score2(mat1,seq1,seq2,score_mat,direc_mat))
    print_pathpair(traceback(direc_mat,seq1,seq2))
    print()

## Align all sequence in the WW-sequence.fasta file

In [12]:
# align all ww-sequence 
for i in range(len(seq_list)):
    for j in range(i+1,len(seq_list)):
        fun(seq_list[i],seq_list[j])

score: 77
VPLPAGWEMAKTSSGQRYFLNHIDQTTTWQDPRK
 ::: :::  .   :. :..::  . : :. :  
SPLPPGWEERQDILGRTYYVNHESRRTQWKRPTP

score: 84
VPLPAGWEMAKTSSGQRYFLNHIDQTTTWQDPRK
 ::: :::    . :. ...::  . : :.::: 
GPLPPGWEERTHTDGRIFYINHNIKRTQWEDPRL
VPLPAGWEMAKT-SSGQRYFLNHIDQTTTWQDPRK
 ::: :::  .: . :. ...::  . : :.::: 
GPLPPGWE-ERTHTDGRIFYINHNIKRTQWEDPRL

score: 75
VPLPAGWEMAKT-SSGQRYFLNHIDQTTTWQDPRK
  :: :::   . :::. :. :::   . :. :  
EKLPPGWEKRMSRSSGRVYYFNHITNASQWERPSG
VPLPAGWE--MAKTSSGQRYFLNHIDQTTTWQDPRK
  :: :::  :.. :::. :. :::   . :. :  
EKLPPGWEKRMSR-SSGRVYYFNHITNASQWERPSG

score: 40
VPLPAGWEMAKTSSGQRYFLNHIDQTTTWQDPRK
    . :   :.  :. :. :   . .::. :  
SGAKSMWTEHKSPDGRTYYYNTETKQSTWEKPDD

score: 45
VPLPAGWEMAKTSSGQRYFLNHIDQTTTWQDPRK
.     :.  :. ::. :. :   . . :  :..
LLSKCPWKEYKSDSGKPYYYNSQTKESRWAKPKE

score: 115
SPLPPGWEERQDILGRTYYVNHESRRTQWKRPTP
 :::::::::    :: .:.::  .::::. :  
GPLPPGWEERTHTDGRIFYINHNIKRTQWEDPRL

score: 99
SPLPPGWEERQD-ILGRTYYVNHESRRTQWKRPTP
  ::::::.:     :: :: :: .  .::.::. 

the sequence : >sp|P46934|892-925  and  >sp|P46934|610-643

score: 115        
SPLPPGWEERQDILGRTYYVNHESRRTQWKRPTP               
GPLPPGWEERTHTDGRIFYINHNIKRTQWEDPRL

gives the maximum score 115. 

They are from the same protein
![](https://ww2.sinaimg.cn/large/006tNbRwgy1fdo7kl0i2pj31340e2acl.jpg)